In [1]:
from _results_helper import *
import warnings
warnings.filterwarnings("ignore")

choices = [
    # "sample",
    "best",
]
layers = [
    # 1,
    2,
    3,
    4,
    # 5,
]
dimensions = [
    # 1,
    # 2,
    # 4,
    8,
    16,
    32,
    # 64,
    # 128,
]

this_file_dir=PosixPath('/home/dzc/code/cvut-colab/results')
Raw logs found. Rewriting data to csv files...


/home/dzc/code/cvut-colab/results/baseline_logs/transport_1_30_r4.log: 100%|██████████| 1500/1500 [00:00<00:00, 4057.32it/s]


skipping hard problems because they take too long


# Train Stats

In [2]:
visualise_train()

blocksworld


ferry


satellite


rover


# Quantified improvements

In [3]:
quantify_performance(choices=choices, layers=layers, dimensions=dimensions)

blocksworld


ferry


satellite


rover


# Relative difference plots

In [4]:
plot_difference(absolute=False, choices=choices, layers=layers, dimensions=dimensions)

blocksworld


ferry


satellite


rover


# Plan length plots

In [5]:
plot_domains("plan_length", log_y=0, choices=choices, layers=layers, dimensions=dimensions)

blocksworld


ferry


satellite


rover


# Cycle plots

In [6]:
plot_domains("cycles", log_y=0, choices=choices, layers=layers, dimensions=dimensions)

blocksworld


ferry


satellite


rover
